In [ ]:
def calculate_grip_balance(df, athlete_id):
    """
    Calculate the grip balance ratio (Arm Side / Glove Side) for a given athlete.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): The ID of the athlete.

    Returns:
        float or None: Grip balance ratio, or None if data is incomplete or missing.
    """
    athlete_data = df[(df['ID'] == athlete_id) & (df['Test Type'] == 'Grip')]
    if athlete_data.empty:
        print(f"No Grip Test data found for athlete ID {athlete_id}.")
        return None

    # Filter by current level
    athlete_level = athlete_data['Level'].iloc[-1]
    peer_data = df[(df['Test Type'] == 'Grip') & (df['Level'] == athlete_level)]

    # Find the highest test number for the athlete
    latest_test_number = athlete_data['Test Number'].max()
    latest_test_data = athlete_data[athlete_data['Test Number'] == latest_test_number]

    arm_side = latest_test_data[latest_test_data['Test Sub-Type'] == 'Arm Side']['Weight'].mean()
    glove_side = latest_test_data[latest_test_data['Test Sub-Type'] == 'Glove Side']['Weight'].mean()

    if pd.isna(arm_side) or pd.isna(glove_side):
        print(f"Incomplete Grip Test data for athlete ID {athlete_id} in latest test.")
        return None

    return arm_side / glove_side

def calculate_shoulder_balance(df, athlete_id):
    """
    Calculate the shoulder balance for a given athlete.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): The ID of the athlete.

    Returns:
        float or None: Shoulder balance, or None if data is incomplete or missing.
    """
    athlete_data = df[(df['ID'] == athlete_id) & 
                      (df['Test Type'] == 'ArmCare') & 
                      (df['Test Sub-Type'] == 'Fresh Exam')]
    if athlete_data.empty:
        print(f"No ArmCare - Fresh Exam data found for athlete ID {athlete_id}.")
        return None

    # Filter by current level
    athlete_level = athlete_data['Level'].iloc[-1]
    peer_data = df[(df['Test Type'] == 'ArmCare') & (df['Test Sub-Type'] == 'Fresh Exam') & (df['Level'] == athlete_level)]

    latest_test_number = athlete_data['Test Number'].max()
    latest_test_data = athlete_data[athlete_data['Test Number'] == latest_test_number]

    shoulder_balance = latest_test_data['Shoulder Balance'].iloc[0]
    if pd.isna(shoulder_balance):
        print(f"Missing Shoulder Balance value for athlete ID {athlete_id} in latest test.")
        return None

    return shoulder_balance

def calculate_facility_grip_balance(df, level):
    """
    Calculate the facility-wide grip balance for a specific level.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        level (str): The competition level to filter by.

    Returns:
        float or None: Facility-wide grip balance, or None if data is missing.
    """
    grip_data = df[(df['Test Type'] == 'Grip') & (df['Level'] == level)]

    def calculate_ratio(group):
        arm_side = group[group['Test Sub-Type'] == 'Arm Side']['Weight'].mean()
        glove_side = group[group['Test Sub-Type'] == 'Glove Side']['Weight'].mean()

        if pd.isna(arm_side) or pd.isna(glove_side):
            return None
        return arm_side / glove_side

    facility_grip_ratios = grip_data.groupby('ID').apply(calculate_ratio).dropna()
    facility_grip_avg = facility_grip_ratios.mean()
    return facility_grip_avg


def calculate_facility_shoulder_balance(df, level):
    """
    Calculate the facility-wide shoulder balance for a specific level.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        level (str): The competition level to filter by.

    Returns:
        float: Facility-wide shoulder balance, or a default value if data is missing.
    """
    shoulder_data = df[(df['Test Type'] == 'ArmCare') & (df['Test Sub-Type'] == 'Fresh Exam') & (df['Level'] == level)]
    if shoulder_data.empty:
        print(f"No facility data for Shoulder Balance at level '{level}'.")
        return 1.00  # Default to balanced if data is missing

    facility_avg_shoulder = shoulder_data['Shoulder Balance'].mean()
    return facility_avg_shoulder

def calculate_jump_balance(df, athlete_id, jump_type):
    """
    Calculate the jump balance ratio for a given athlete.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): The ID of the athlete.
        jump_type (str): The type of jump ("Lateral" or "Vertical").

    Returns:
        float or None: Jump balance ratio, or None if data is incomplete or missing.
    """
    athlete_data = df[(df['ID'] == athlete_id) & (df['Test Type'] == "Jump")]

    if athlete_data.empty:
        return None

    athlete_level = athlete_data['Level'].iloc[-1]
    peer_data = df[(df['Test Type'] == 'Jump') & (df['Level'] == athlete_level)]

    block_col, load_col = ("Lateral Block Leg", "Lateral Load Leg") if jump_type == "Lateral" else ("Vertical Block Leg", "Vertical Load Leg")
    metric_col = "Distance" if jump_type == "Lateral" else "Vert"

    block_data = athlete_data[athlete_data['Test Sub-Type'] == block_col]
    load_data = athlete_data[athlete_data['Test Sub-Type'] == load_col]

    if block_data.empty or load_data.empty:
        return None

    block_leg = block_data[metric_col].iloc[-1] if not block_data[metric_col].isna().all() else None
    load_leg = load_data[metric_col].iloc[-1] if not load_data[metric_col].isna().all() else None

    if block_leg is None or load_leg is None:
        return None

    return block_leg / load_leg

def calculate_facility_jump_balance(df, jump_type, level):
    """
    Calculate the facility-wide jump balance for a specific jump type and level.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        jump_type (str): The type of jump ("Lateral" or "Vertical").
        level (str): The competition level to filter by.

    Returns:
        float or None: Facility-wide jump balance, or None if data is incomplete or missing.
    """
    block_col, load_col = ("Lateral Block Leg", "Lateral Load Leg") if jump_type == "Lateral" else ("Vertical Block Leg", "Vertical Load Leg")
    metric_col = "Distance" if jump_type == "Lateral" else "Vert"

    facility_data = df[(df['Test Sub-Type'] == block_col) & (df['Level'] == level)]
    facility_block = facility_data[metric_col].mean()

    facility_data = df[(df['Test Sub-Type'] == load_col) & (df['Level'] == level)]
    facility_load = facility_data[metric_col].mean()

    if facility_block > 0 and facility_load > 0:
        return facility_block / facility_load
    return None